In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# # Testing to ensure GPU is being utilized.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np

import os
from zipfile import ZipFile

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D, MaxPooling2D ,Flatten
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


np.random.seed(42)
tf.random.set_seed(42)

# Import dataset

In [ ]:
zippath="/content/drive/MyDrive/Age_Estimation/Dataset/combined_faces_train_augmented_rottaion.zip"

with ZipFile(zippath,"r") as data:
  data.extractall()
print("unzip done")

In [ ]:
zippath="/content/drive/MyDrive/Age_Estimation/Dataset/combined_faces_refined_final.zip"

with ZipFile(zippath,"r") as data:
  data.extractall()
print("unzip done")

# Read the Data

In [ ]:
df_aug_trainset=pd.read_csv("/content/drive/MyDrive/Age_Estimation/csv/images_filenames_labels_train_augmented_mega.csv")
print(df_aug_trainset.shape)
df_aug_trainset.head()

In [ ]:
df_test_refined=pd.read_csv("/content/drive/MyDrive/Age_Estimation/csv/images_filenames_labels_test_final.csv")
print(df_test_refined.shape)
df_test_refined.head()

In [ ]:
df_aug_trainset=shuffle(df_aug_trainset,random_state=42).reset_index(drop=True)
df_aug_trainset.head()

# Create tensorflow Dataset

In [ ]:
train_filename_tensor=tf.constant(list(df_aug_trainset["filename"]))
train_label_tensor=tf.constant(list(df_aug_trainset["age_class"]))

test_filename_tensor=tf.constant(list(df_test_refined["filename"]))
test_label_tensor=tf.constant(list(df_test_refined["age_class"]))

In [ ]:
num_classes = 7

def parse_function(filename, label):

    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)    # channels=1 to convert to grayscale, channels=3 to convert to RGB.
    image_decoded = tf.image.resize(image_decoded, [224, 224])
    label = tf.one_hot(label, num_classes)

    return image_decoded, label

In [ ]:
train_tf_dataset=tf.data.Dataset.from_tensor_slices((train_filename_tensor,train_label_tensor)).map(parse_function).batch(512)
test_tf_dataset=tf.data.Dataset.from_tensor_slices((test_filename_tensor,test_label_tensor)).map(parse_function).batch(512)


# Define Model Architecture

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
from keras.models import Model

base_model=MobileNet( weights='imagenet',include_top=False,input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)

x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3

preds=Dense(7,activation='softmax')(x)

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
tensorboard = TensorBoard(log_dir=f"/content/drive/MyDrive/Age_Estimation/log_final/")
checkpoint = ModelCheckpoint(filepath=f"/content/drive/MyDrive/Age_Estimation/log_final/model_checkpoint_reassignedAge_mobilenet.h5",
                             monitor='val_accuracy',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )

# Train the model

In [ ]:
history=model.fit(train_tf_dataset,batch_size=512,
                        validation_data=test_tf_dataset,
                        epochs=30, callbacks=[tensorboard,checkpoint],
                        shuffle=True)

# Visualize the training process

In [ ]:
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

In [ ]:
# Plotting a line chart to visualize the loss and accuracy values by epochs.

import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)


# Evaluate model

In [ ]:

final_cnn_score = model.evaluate(test_tf_dataset, verbose=1)


In [ ]:
from google.colab import runtime
runtime.unassign()